- ### ne1_hr_lc
  Satellite-derived land cover data and shaded relief presented with a light, natural palette suitable for making thematic and reference maps. Two versions of the 10 million-scale raster data are offered: high resolution files at 21,600 x 10,800 pixels and low resolution at 16,200 x 8,100. The raster files register precisely with the 10m vector data. 

  More information: https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/raster/NE1_HR_LC.zip

In [ ]:
import os, re
from IPython.display import display_html, display_markdown

import numpy as np
import pandas as pd

CONNECTION_STRING = os.getenv('AWSGPDBCONN')

cs = re.match('^postgresql:\/\/(\S+):(\S+)@(\S+):(\S+)\/(\S+)$', CONNECTION_STRING)

DB_USER   = cs.group(1)
DB_PWD    = cs.group(2)
DB_SERVER = cs.group(3)
DB_PORT   = cs.group(4)
DB_NAME   = cs.group(5)
con = CONNECTION_STRING 

%reload_ext sql
%sql $CONNECTION_STRING

In [ ]:
%%sql $DB_USER@$DB_SERVER
SELECT UNNEST(ARRAY[version, postgis_full_version]) version_info FROM (SELECT version()) A, (SELECT postgis_full_version()) B

### Scratch pad

%%sql
SELECT COUNT(*) AS cnt, 'tiger.addr' AS tblname FROM tiger.addr UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.addrfeat' AS tblname FROM tiger.addrfeat UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.bg' AS tblname FROM tiger.bg UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.county' AS tblname FROM tiger.county UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.county_lookup' AS tblname FROM tiger.county_lookup UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.countysub_lookup' AS tblname FROM tiger.countysub_lookup UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.cousub' AS tblname FROM tiger.cousub UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.direction_lookup' AS tblname FROM tiger.direction_lookup UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.edges' AS tblname FROM tiger.edges UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.faces' AS tblname FROM tiger.faces UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.featnames' AS tblname FROM tiger.featnames UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.geocode_settings' AS tblname FROM tiger.geocode_settings UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.geocode_settings_default' AS tblname FROM tiger.geocode_settings_default UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.loader_lookuptables' AS tblname FROM tiger.loader_lookuptables UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.loader_platform' AS tblname FROM tiger.loader_platform UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.loader_variables' AS tblname FROM tiger.loader_variables UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.pagc_gaz' AS tblname FROM tiger.pagc_gaz UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.pagc_lex' AS tblname FROM tiger.pagc_lex UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.pagc_rules' AS tblname FROM tiger.pagc_rules UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.place' AS tblname FROM tiger.place UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.place_lookup' AS tblname FROM tiger.place_lookup UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.secondary_unit_lookup' AS tblname FROM tiger.secondary_unit_lookup UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.state' AS tblname FROM tiger.state UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.state_lookup' AS tblname FROM tiger.state_lookup UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.street_type_lookup' AS tblname FROM tiger.street_type_lookup UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.tabblock' AS tblname FROM tiger.tabblock UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.tract' AS tblname FROM tiger.tract UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.zip_lookup' AS tblname FROM tiger.zip_lookup UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.zip_lookup_all' AS tblname FROM tiger.zip_lookup_all UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.zip_lookup_base' AS tblname FROM tiger.zip_lookup_base UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.zip_state' AS tblname FROM tiger.zip_state UNION ALL 
SELECT COUNT(*) AS cnt, 'tiger.zip_state_loc' AS tblname FROM tiger.zip_state_loc

SELECT g.rating, ST_AsText(ST_SnapToGrid(g.geomout,0.00001)) As wktlonlat,
(addy).address As stno, (addy).streetname As street,
(addy).streettypeabbrev As styp, (addy).location As city, (addy).stateabbrev As st,(addy).zip

%%sql
SELECT rid
    , (md).* 
FROM (
    SELECT rid
        , ST_MetaData(rast) AS md 
    FROM public.ne1_hr_lc
) AS foo 
ORDER BY random() 
LIMIT 25;

%%sql
DROP TABLE IF EXISTS public.uk_raster;
CREATE TABLE public.uk_raster AS
SELECT r.rast
    , c.*
FROM public.ne1_hr_lc AS r
    , public.ne_10m_admin_0_countries AS c
WHERE ST_Intersects(r.rast, ST_Transform(c.geom, 4326)) 
    AND c.name = 'United Kingdom';
    
SELECT * FROM public.uk_raster;

%%sql
SELECT ST_GDALDrivers();

%%sql
CREATE TEMP TABLE tmp_out AS
SELECT lo_from_bytea(0,
       ST_AsGDALRaster(ST_Union(rast), 'GTiff',  ARRAY['COMPRESS=DEFLATE', 'PREDICTOR=2', 'PZLEVEL=9'])
        ) AS loid
FROM public.uk_raster;

SELECT lo_export(loid, '/root/github/gp-explained/NE1_HR_LC/uk_raster.tif')
FROM public.tmp_out;

import rasterio
import rasterio.plot
from rasterio.windows import Window
import pyproj
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

filepath = '/root/github/gp-explained/NE1_HR_LC/NE1_HR_LC.tif'

dataset = rasterio.open(filepath)

print(dataset.count)
print(dataset.width, dataset.height)
print(dataset.bounds)

dataset.indexes

dataset.read(3)

with rasterio.open(filepath) as src:
--   oviews = src.overviews(1) # list of overviews from biggest to smallest
--  oview = oviews[-1] # let's look at the smallest thumbnail
--   print('Decimation factor= {}'.format(oview))
   -- NOTE this is using a 'decimated read' (http://rasterio.readthedocs.io/en/latest/topics/resampling.html)
   thumbnail = src.read(1, out_shape=(1, int(src.height // 32), int(src.width // 32)))

print('array type: ',type(thumbnail))
print(thumbnail)

plt.imshow(thumbnail)
plt.colorbar()
plt.title('Overview - Band 4 {}'.format(thumbnail.shape))
plt.xlabel('Column #')
plt.ylabel('Row #')